In [1]:
import transformers
import pandas as pd
import re

import sys
sys.path.insert(0,"../..")
from utils.Clean_Utils import *

clean_utils = Clean_Utils()

/Users/guneet/opt/miniconda3/envs/bdproj/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [106]:
# from transformers import AutoTokenizer,DataCollatorWithPadding,XLMRobertaModel, XLMRobertaTokenizer
# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [2]:
# df_mh = pd.read_csv('../../data/consolidatedmhlong.csv')
# df_nmh = pd.read_csv('../../data/consolidatednmhlong.csv')
df_mh = pd.read_csv('../../data/consolidatedmh.csv')
df_nmh = pd.read_csv('../../data/consolidatednmh.csv')

In [3]:
print(len(df_mh),len(df_nmh))

232975 255527


In [4]:
# df_mh['Text'] = df_mh['Title'].astype('str') + df_mh['Text'].astype('str')
# df_nmh['Text'] = df_nmh['Title'].astype('str') + df_nmh['Text'].astype('str')

df_mh['Text'] =  df_mh['Text'].astype('str')
df_nmh['Text'] =  df_nmh['Text'].astype('str')

In [5]:
df_mh = clean_utils.keep_words_gt_n(df_mh,10)
df_nmh = clean_utils.keep_words_gt_n(df_nmh,10)

In [6]:
print(len(df_mh),len(df_nmh))

174241 142756


Need to address imbalance, by pulling more data for the minority class

In [7]:
sample = df_mh['Text'].iloc[:10].tolist()

In [8]:
for i,v in enumerate(sample):
    print(i,v)
    print()

0 About an hour and a half after I have taken my meds (olanzapine) the restless in my legs and arms begin. But not every day. Maybe four times a week on average. I try to move a lot, and shake it off, but it keeps coming back and it makes it impossible to sleep. I try to postpone going to bed and try to relax with a series on telly but I can’t focus because I have to move my arms and legs. So I’ve tried to take my meds later so that I might fall asleep before the restlessness comes but then the next day is ruined because I can’t get up in the morning and I end up like a zombie way into the day. 

So thing is, I don’t have a lot of symptoms at the moment and I’ve been stable for a long time. Should I ask the doctor if I can stop taking the ap. I want to because I’m struggling with this restlessness, but I’m afraid I might end up psychotic again. Changing meds is out of the picture. I’ve tried so many. 

Not really looking for an answer here, just had to type it out.

1 Can psycosis end 

Looks like the text strings have many new line escape characters and also contains link, we should remove them.

In [9]:
sample[8]

'Attached below is todays video link to my “On Conquering Schizophrenia” YouTube channel. Today entails the mechanism of reinforcement. Like all, todays video is ever brief and can be viewed amid a certain assurance! \n\nhttps://youtu.be/YYKVQtHYbnU'

In [10]:
clean_utils.remove_links((clean_utils.remove_newline(sample[8])))

'Attached below is todays video link to my “On Conquering Schizophrenia” YouTube channel. Today entails the mechanism of reinforcement. Like all, todays video is ever brief and can be viewed amid a certain assurance! .'

In [11]:
sample[0]

'About an hour and a half after I have taken my meds (olanzapine) the restless in my legs and arms begin. But not every day. Maybe four times a week on average. I try to move a lot, and shake it off, but it keeps coming back and it makes it impossible to sleep. I try to postpone going to bed and try to relax with a series on telly but I can’t focus because I have to move my arms and legs. So I’ve tried to take my meds later so that I might fall asleep before the restlessness comes but then the next day is ruined because I can’t get up in the morning and I end up like a zombie way into the day. \n\nSo thing is, I don’t have a lot of symptoms at the moment and I’ve been stable for a long time. Should I ask the doctor if I can stop taking the ap. I want to because I’m struggling with this restlessness, but I’m afraid I might end up psychotic again. Changing meds is out of the picture. I’ve tried so many. \n\nNot really looking for an answer here, just had to type it out.'

In [12]:
clean_utils.remove_links((clean_utils.remove_newline(sample[0])))

'About an hour and a half after I have taken my meds (olanzapine) the restless in my legs and arms begin. But not every day. Maybe four times a week on average. I try to move a lot, and shake it off, but it keeps coming back and it makes it impossible to sleep. I try to postpone going to bed and try to relax with a series on telly but I can’t focus because I have to move my arms and legs. So I’ve tried to take my meds later so that I might fall asleep before the restlessness comes but then the next day is ruined because I can’t get up in the morning and I end up like a zombie way into the day. .So thing is, I don’t have a lot of symptoms at the moment and I’ve been stable for a long time. Should I ask the doctor if I can stop taking the ap. I want to because I’m struggling with this restlessness, but I’m afraid I might end up psychotic again. Changing meds is out of the picture. I’ve tried so many. .Not really looking for an answer here, just had to type it out.'

In [13]:
df_mh['Text'] = df_mh['Text'].apply(clean_utils.remove_links).apply(clean_utils.remove_newline)
df_nmh['Text'] = df_nmh['Text'].apply(clean_utils.remove_links).apply(clean_utils.remove_newline)

In [14]:
for i,v in enumerate(df_mh['Text'].iloc[:10].tolist()):
    print(i,v)
    print()

0 About an hour and a half after I have taken my meds (olanzapine) the restless in my legs and arms begin. But not every day. Maybe four times a week on average. I try to move a lot, and shake it off, but it keeps coming back and it makes it impossible to sleep. I try to postpone going to bed and try to relax with a series on telly but I can’t focus because I have to move my arms and legs. So I’ve tried to take my meds later so that I might fall asleep before the restlessness comes but then the next day is ruined because I can’t get up in the morning and I end up like a zombie way into the day. .So thing is, I don’t have a lot of symptoms at the moment and I’ve been stable for a long time. Should I ask the doctor if I can stop taking the ap. I want to because I’m struggling with this restlessness, but I’m afraid I might end up psychotic again. Changing meds is out of the picture. I’ve tried so many. .Not really looking for an answer here, just had to type it out.

1 Can psycosis end in

In [15]:
df_nmh['label']=0
df_mh['label']=1

In [16]:
from datasets import Dataset, DatasetDict
dataset_cols=["Text","label"]

In [17]:
dataset_nmh = df_nmh[dataset_cols]
dataset_mh = df_mh[dataset_cols]

In [18]:
dataset_nmh = dataset_nmh.sample(frac=1).reset_index().drop('index',axis=1)
dataset_mh = dataset_mh.sample(frac=1).reset_index().drop('index',axis=1)

In [19]:
# train_nmh = dataset_nmh[:230000]
# train_mh = dataset_mh[:230000]

# eval_nmh = dataset_nmh[230000:240000]
# eval_mh = dataset_mh[230000:240000]

# test_nmh = dataset_nmh[240000:]
# test_mh = dataset_mh[240000:]

train_nmh = dataset_nmh[:140000]
train_mh = dataset_mh[:140000]

eval_nmh = dataset_nmh[140000:142000]
eval_mh = dataset_mh[140000:142000]

test_nmh = dataset_nmh[142000:]
test_mh = dataset_mh[142000:]

In [20]:
dataset = DatasetDict({
    'train': Dataset.from_pandas(pd.concat([train_mh,train_nmh]).sample(frac=1)), 
    'test': Dataset.from_pandas(pd.concat([test_mh,test_nmh]).sample(frac=1)), 
    'validation': Dataset.from_pandas(pd.concat([eval_mh,eval_nmh]).sample(frac=1))
})

In [21]:
# dataset.save_to_disk("ww-binary-dataset-small")
dataset.save_to_disk("ww-binary-dataset")

In [131]:
dataset['train']

Dataset({
    features: ['Text', 'label', '__index_level_0__'],
    num_rows: 460000
})

In [22]:
pd.concat([train_mh,train_nmh]).sample(frac=1).to_csv('train.csv')
pd.concat([test_mh,test_nmh]).sample(frac=1).to_csv('test.csv')
pd.concat([eval_mh,eval_nmh]).sample(frac=1).to_csv('eval.csv')